## 路径推理

In [1]:
using StringDistances
using DataStructures

In [1]:
include("../src/translatedata.jl")
include("../src/deduction.jl")
mkpath("ILLs")

LoadError: LoadError: UndefVarError: nearby not defined
in expression starting at /home/rex/work_space/7 others/ccks/CCKS-mKGQA/src/deduction.jl:13

In [3]:
count(∈(Set(train_ques)), valid_ques)

592

### 初始化

In [5]:
# 初始化查询字典
train_rels = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for sols in unique(train_ques_sols)
    que = first(sols)
    rels = Tuple(@view(sols[3:end]))
    rels ∉ train_rels[que] && push!(train_rels[que], rels)
end
"匹配最近 10 个问题"
top10 = nearby(unique(train_ques))

top10

### 示例-方刚

In [7]:
# 步骤一
# 获取 top 10 匹配问题以及分数
que, ner = valid_ques_ner[1]
ques, scores = top10(que)
println(scores)
println(que)
println(ques[1])
# 获取问题中的关系
rels = vcat([train_rels[que] for que in ques]...)

[0, 8, 9, 16, 16, 16, 17, 17, 17, 17]
where is the constituency of the one who is alongside NER from?
where is the constituency of the one who is alongside NER from?


10-element Vector{Tuple}:
 ("alongside", "constituency")
 ("alongside", "constituency")
 ("alongside", "constituency")
 ("alongside", "birthplace")
 ("alongside", "almamater")
 ("alongside", "title")
 ("alongside", "title")
 ("alongside", "almamater")
 ("leader", "constituency")
 ("alongside", "office")

In [8]:
# 计算重数，确认两跳关系为 **alongside, constituency**
println(counter(first.(rels)))
counter(last.(rels))

Accumulator(leader => 1, alongside => 9)


Accumulator{SubString{String}, Int64} with 5 entries:
  "constituency" => 4
  "office"       => 1
  "almamater"    => 2
  "title"        => 2
  "birthplace"   => 1

In [9]:
# 第一跳精确匹配
_, _, obj = filter(i->i[1]==ner && i[2]=="alongside", triples)[1]

("felix_chung", "alongside", "vincent_fang_(entrepreneur)")

In [10]:
# 第二跳模糊匹配
possible = unique!(first.(filter(i->i[2]=="constituency", triples)))
println("第二跳可选的实体总数: ", length(possible))
res, scos = nearby(possible, obj; char=false)
# res, scos = nearby(unique!(first.(triples)), obj; char=false) # 对照
println(scos)
res

第二跳可选的实体总数: 133
[2, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807]


10-element Vector{String}:
 "fang_gang"
 "liu_huangfa"
 "goldstein"
 "guo_weiqiang"
 "lu_weiguo"
 "he_qiming_(federation_of_trade_unions)"
 "huang_zhenxia"
 "law_cheung-kwok"
 "he_shizhu"
 "ma_li_(dab_chairman)"

### 示例-周武帝

In [11]:
# 步骤一
# 获取 top 10 匹配问题以及分数
que, ner = "who is before the father of NER?", "princess_of_qingdu"
ques, scores = top10(que)
# 获取问题中的关系，精确匹配
rels = vcat([train_rels[que] for que in ques]...)
rel1 = first(sort_count(counter(first.(rels))))[1]
rel2 = first(sort_count(counter(last.(rels))))[1]
ner, rel1, rel2

("princess_of_qingdu", "father", "before")

In [12]:
# 第一跳精确匹配
_, _, obj = filter(i->i[1]==ner && i[2]=="father", triples)[1]

("princess_of_qingdu", "father", "zhou_wudi")

In [14]:
obj = "emperor_wu_of_zhou"

"emperor_wu_of_zhou"

In [15]:
# 第二跳模糊匹配
possible = unique!(first.(filter(i->i[2]=="before", triples)))
println(length(possible))
res, scos = nearby(possible, obj; char=false)
println(scos)
res

4924
[1, 2, 2, 2, 2, 2, 2, 2, 2, 2]


10-element Vector{String}:
 "emperor_wu_of_northern_zhou"
 "emperor_yingzong_of_song"
 "emperor_gaozu_of_tang"
 "emperor_hui_of_jin"
 "emperor_wu_of_southern_qi"
 "emperor_xuanzong_of_tang"
 "emperor_ruizong_of_tang"
 "emperor_taizong_of_song"
 "emperor_daizong_of_tang"
 "emperor_xuan_of_northern_zhou"

In [ ]:
# 翻译改用 "emperor_wu_of_zhou"
obj = "emperor_wu_of_zhou"

模型特点：
- 第一跳正确性很高
- 实体对齐会影响推理准确性
- 匹配度低时，可能出现**关系**被替换的现象

### 错误匹配

In [74]:
que, ner = valid_ques_ner[13]
ques, scores = top10(que)
println(que)
println(ques[1])
rels = vcat([train_rels[que] for que in ques]...)

what is the type of the people, things and things with the same name as NER?
what is the rank of the people, things and things with the same name as NER?


10-element Vector{Tuple}:
 ("shipnamesake", "rank")
 ("shipnamesake", "father")
 ("shipnamesake", "mother")
 ("shipnamesake", "influenced")
 ("shipnamesake", "largestcityname")
 ("shipnamesake", "largestcityname")
 ("shipnamesake", "capital")
 ("shipnamesake", "politicalsystem")
 ("shipnamesake", "father")
 ("shipnamesake", "influences")

In [77]:
edges[ner]

2-element Vector{Tuple}:
 ("shipnamesake", "mauna_kea")
 ("title", "united_states_navy")

### 无法识别的例子

In [78]:
que, ner = valid_ques_ner[5]
ques, scores = top10(que)
println(que)
println(ques[1])
rels = vcat([train_rels[que] for que in ques]...)

which draft team does the author of NER dj quik lord tariq and peter gunz belong to?
which state does the client of NER verenigde staten van amerika belong to?


10-element Vector{Tuple}:
 ("client", "state")
 ("parents", "title")
 ("location", "south", "state")
 ("father", "house")
 ("location", "tenants", "sport")
 ("chronology", "label")
 ("eponym", "ethnicity")
 ("location", "east", "constituencywestminster")
 ("related", "type")
 ("related", "title")

In [79]:
sort_count(counter(first.(rels)))

OrderedDict{SubString{String}, Int64} with 7 entries:
  "location"   => 3
  "related"    => 2
  "father"     => 1
  "parents"    => 1
  "eponym"     => 1
  "client"     => 1
  "chronology" => 1

In [80]:
edges[ner]

1-element Vector{Tuple}:
 ("chronology", "shaquille_o'neal")

In [75]:
# 通过词频察觉错误，没有明显梯度
sort_count(counter(last.(rels)))

OrderedDict{SubString{String}, Int64} with 8 entries:
  "father"          => 2
  "largestcityname" => 2
  "rank"            => 1
  "influenced"      => 1
  "capital"         => 1
  "mother"          => 1
  "politicalsystem" => 1
  "influences"      => 1

倾向采用常规模型

In [76]:
# 实际答案
edges = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (sub, rel, obj) in triples
    push!(edges[sub], (rel, obj))
end
edges[ner]

2-element Vector{Tuple}:
 ("shipnamesake", "mauna_kea")
 ("title", "united_states_navy")

In [ ]:
_, _, obj = filter(i->i[1]==ner && i[2]=="shipnamesake", triples)[1]
edges[obj]

In [ ]:
"what is the type of the people, things and things with the same name as NER?	uss_mauna_kea_(ae-22)	title	commander1label	style"

In [ ]:
count(length(edges[ner]) > 1 for (_, ner) in valid_ques_ner)

### 实体对齐

1. 提取从 zh/en 三元组的实体，这是目标要做的
   ```jl
   en1
   en2
   ...
   zh1
   zh2
   ...
   ```
2. 从 ILLs 中筛选有价值的部分:
    - A \neq B
    - A or B in triples
    此时行数减少，比如
    ```jl
    en1 
    en2 zh1
    ...
    zh2
    zh3
    ...
    ```
3. 中英文均进行翻译，补充到数据中，其中中文翻译两次
4. 使用相似匹配，进一步缩小数据长度（忽略大小写，空格，指定符号）

### 处理 MergeILLs-暂存

In [ ]:
function get_triple_sign(line)
    triple = split(strip(line), ' ')
    subsign, relsign, objsign = "en", "en", "en"
    sub = get_txt(en_obj_reg, triple[1])
    rel = get_txt(en_rel_reg, triple[2])
    obj = get_txt(en_obj_reg, triple[3])
    isempty(sub) && (sub = get_txt(zh_obj_reg, triple[1]);subsign = "zh")
    isempty(rel) && (rel = get_txt(zh_rel_reg, triple[2]);relsign = "zh")
    isempty(obj) && (obj = get_txt(zh_obj_reg, triple[3]);objsign = "zh")
    [(sub, rel, obj), (subsign, relsign, objsign)]
end
function sort_count(list)
    c_list = counter(list)
    sort(Dict(c_list), by=i->c_list[i], rev=true)
end

In [ ]:
# 读取数据
merged_triples = NTuple{3, String}[]
signs = NTuple{3, String}[]
open("ILLs_fusion/merged_ILLs_KG_en_zh.txt", "r") do io
    while true
        line = readline(io)
        isempty(line) && break
        triple, sign = get_triple_sign(line)
        push!(merged_triples, triple)
        push!(signs, sign)
    end
end
zh_triples_ILLs = merged_triples[162545:end]
zh_signs = signs[162545:end]

# 写入数据
# extract_io = open("extract/merged_en_zh_byILLs.txt", "w")
# open("ILLs_fusion/merged_ILLs_KG_en_zh.txt", "r") do io
#     while (line = readline(io)) != ""
#         triple, sign = get_triple_sign(line)
#         println(extract_io, join(triple, '\t'))
#     end
# end

In [ ]:
# 中文存在一对多的情况
downloadILLs = Tuple{String, String}[]
for (raw, new, sign) in zip(zh_triples, zh_triples_ILLs, zh_signs)
    new[1] == new[3] && continue
    sign[1] == "en" && new[1] != raw[1] && push!(downloadILLs, (new[1], raw[1]))
    sign[3] == "en" && new[3] != raw[3] && push!(downloadILLs, (new[3], raw[3]))
end
unique!(downloadILLs) # 58556

In [ ]:
# 合并后的实体对齐
open("ILLs/combine_ills.txt", "w") do io
    for (en, zh) in sort(unique!(vcat(downloadILLs, ILLs)), by=last)
        println(io, en, '\t', zh)
    end
end

In [ ]:
# 不在 ILLs 中的实体对齐
open("ILLs/new_ills.txt", "w") do io
    for (en, zh) in sort(filter(∉(Set(ILLs)), downloadILLs), by=last)
        println(io, en, '\t', zh)
    end
end

In [ ]:
dlILLs = Dict{String, String}()
en_subs = Set(first.(en_triples))
for ele in zip(zh_triples, zh_triples_ILLs, signs), (raw, new, sign) in zip(ele...)
    sign == "en" && isascii(new) && (dlILLs[raw] = new)
end

In [ ]:
fails = Tuple{String, String}[]
for (key, val) in ILLs_zh_en
    (haskey(dlILLs, key) && dlILLs[key] == val) || push!(fails, (key, val))
end

In [ ]:
en_objs = Set(last.(en_triples))
en_subs = Set(first.(en_triples))

In [ ]:
necc_ILLs, necc_triples = Set{Tuple{String, String}}(), NTuple{3, String}[]
for (raw, new, sign) in zip(zh_triples, zh_triples_ILLs, signs)
    # 英文实体，头接尾，尾接头
    flag = false
    if sign[1] == "en" && !isascii(raw[1]) && new[1] ∈ en_objs
        tmp = (raw[1], new[1])
        tmp ∈ necc_ILLs || (push!(necc_ILLs, tmp); flag=true)
    end
    if sign[3] == "en" && !isascii(raw[3]) && new[3] ∈ en_subs
        tmp = (raw[3], new[3])
        tmp ∈ necc_ILLs || (push!(necc_ILLs, tmp); flag=true)
    end
    flag && push!(necc_triples, raw)
end

In [ ]:
words = unique!(vcat(first.(zh_triples), last.(zh_triples_ILLs)))
println("中文实体总数:\t", length(words))
non_ILLs = filter(∉(keys(ILLs_zh_en)), words)
println("非 ILLs 对齐实体数目:\t", length(non_ILLs))


In [ ]:
necc_ILLs
"约翰内斯堡大学" => "University of Johannesburg"
"華特迪士尼工作室電影" => "Walt Disney Studios Electric"

In [ ]:
for (key, val) in fails
    haskey(dlILLs, key) && println(val, '\t', dlILLs[key])
end